In [1]:
!pip install transformers==4.44.2 joblib==1.4.2 scikit-learn==1.6.0 numpy==1.26.4 pandas==2.2.3 scipy==1.13.1 seaborn==0.13.2 tqdm==4.66.5 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 72.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 83.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 44.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 83.1 MB/s eta 0:00:00:00:01
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.67.1
    Uninstalling tqdm-4.67.1:
      Successfully uninstalled tqdm-4.67.1
  Attempting uninstall

In [2]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
import torch
from torch.utils.data import DataLoader, Dataset


# Load datasets
train_df = pd.read_csv('/kaggle/input/pampa-set/Train.csv')
train_df = train_df[['ID', 'SMILES', 'Permeability']]
test_df = pd.read_csv('/kaggle/input/pampa-set/Test.csv')
test_df = test_df[['ID', 'SMILES', 'Permeability']]

In [3]:
tokenizer = AutoTokenizer.from_pretrained("seyonec/PubChem10M_SMILES_BPE_450k")
model = AutoModelForSequenceClassification.from_pretrained('seyonec/PubChem10M_SMILES_BPE_450k', num_labels=1)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

tokenizer_config.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/515 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/336M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at seyonec/PubChem10M_SMILES_BPE_450k and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
# Custom dataset class
class SMILESDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=325):
        self.tokenizer = tokenizer
        self.dataframe = dataframe
        self.max_length = max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        smiles = self.dataframe.iloc[idx]['SMILES']
        permeability = self.dataframe.iloc[idx]['Permeability']
        inputs = self.tokenizer(smiles, return_tensors='pt', padding="max_length", truncation=True, max_length=self.max_length)
        
        input_ids = inputs['input_ids'].squeeze(0)  # Shape: (sequence_length,)
        attention_mask = inputs['attention_mask'].squeeze(0)  # Shape: (sequence_length,)
        
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': torch.tensor(permeability, dtype=torch.float)
        }

In [5]:
# datasets
train_dataset = SMILESDataset(train_df, tokenizer)
test_dataset = SMILESDataset(test_df, tokenizer)
batch_size = 16
# data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [6]:
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
num_epochs = 20

In [7]:
# Training loop
from tqdm import tqdm
for epoch in range(num_epochs):
    print(f"Entered Epoch {epoch + 1}")
    model.train()
    train_loss = 0

    for batch in tqdm(train_loader, desc=f'Training Epoch {epoch + 1}/{num_epochs}', unit='batch'):
        optimizer.zero_grad()

        # Move all batch tensors to device
        batch = {k: v.to(device) for k, v in batch.items()}
        labels = batch["labels"].unsqueeze(1)  # still shape: (batch_size, 1)

        # Forward pass
        outputs = model(
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"],
            labels=labels
        )
        loss = outputs.loss
        train_loss += loss.item()

        # Backprop and optimizer step
        loss.backward()
        optimizer.step()

    avg_train_loss = train_loss / len(train_loader)
    print(f'Epoch {epoch + 1}/{num_epochs} - Train Loss: {avg_train_loss:.4f}')

Entered Epoch 1


Training Epoch 1/20: 100%|██████████| 348/348 [02:45<00:00,  2.11batch/s]


Epoch 1/20 - Train Loss: 0.8255
Entered Epoch 2


Training Epoch 2/20: 100%|██████████| 348/348 [02:47<00:00,  2.08batch/s]


Epoch 2/20 - Train Loss: 0.4076
Entered Epoch 3


...

Epoch 18/20 - Train Loss: 0.1395
Entered Epoch 19


Training Epoch 19/20: 100%|██████████| 348/348 [02:47<00:00,  2.08batch/s]


Epoch 19/20 - Train Loss: 0.1338
Entered Epoch 20


Training Epoch 20/20: 100%|██████████| 348/348 [02:47<00:00,  2.08batch/s]

Epoch 20/20 - Train Loss: 0.1290


In [8]:
# Saving the model after training
model_name = 'PubChem10M_SMILES_BPE_450k_model_1_pampa'
model_save_path = f'/kaggle/working/{model_name}'
os.makedirs(model_save_path, exist_ok=True)

tokenizer.save_pretrained(model_save_path)
model.save_pretrained(model_save_path)

print(f'Model and tokenizer saved to {model_save_path}')

Model and tokenizer saved to /kaggle/working/PubChem10M_SMILES_BPE_450k_model_1_pampa


In [9]:
from scipy.stats import pearsonr, spearmanr

model.eval()
test_loss = 0
test_true_labels = []
predictions = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc='Testing', unit='batch'):
      
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].unsqueeze(1).to(device).float()

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        test_loss += loss.item()

        test_true_labels.extend(labels.cpu().numpy())
        preds = outputs.logits.squeeze().cpu().numpy()  
        predictions.extend(preds)

# Final test loss
avg_test_loss = test_loss / len(test_loader)
print(f'Test Loss: {avg_test_loss:.4f}')

test_true_labels = np.array(test_true_labels).flatten()
predictions = np.array(predictions)
print(test_true_labels.shape)
print(predictions.shape)

# Performance metrics
mse = mean_squared_error(test_true_labels, predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(test_true_labels, predictions)
r2 = r2_score(test_true_labels, predictions)
PCC,_ = pearsonr(test_true_labels, predictions)
SCC,_ = spearmanr(test_true_labels, predictions)

# Print performance metrics
print(f'Mean Squared Error: {mse:.4f}')
print(f'Root Mean Squared Error: {rmse:.4f}')
print(f'Mean Absolute Error: {mae:.4f}')
print(f'R^2 Score: {r2:.4f}')
print(f'Pearson Correlation Coefficient: {PCC:.4f}')
print(f'Spearman Correlation Coefficient: {SCC:.4f}')

# Print hyperparameters
print("Hyperparameters:")
print(f"Learning Rate: {5e-5}")
print(f"Batch Size: 16")
print(f"Epochs: {num_epochs}")

Testing: 100%|██████████| 87/87 [00:14<00:00,  6.08batch/s]


Test Loss: 0.3029
(1392,)
(1392,)
Mean Squared Error: 0.3029
Root Mean Squared Error: 0.5504
Mean Absolute Error: 0.3849
R^2 Score: 0.5213
Pearson Correlation Coefficient: 0.7553
Spearman Correlation Coefficient: 0.7345
Hyperparameters:
Learning Rate: 5e-05
Batch Size: 16
Epochs: 20


In [10]:
import os
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from transformers import AutoTokenizer, AutoModel

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

model_name = 'PubChem10M_SMILES_BPE_450k_model_1_pampa'
model_save_path = f'/kaggle/working/{model_name}'

if not os.path.exists(model_save_path):
    raise FileNotFoundError(f"The model directory {model_save_path} does not exist.")

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_save_path, trust_remote_code=True)
model = AutoModel.from_pretrained(model_save_path, trust_remote_code=True).to(device)

Some weights of RobertaModel were not initialized from the model checkpoint at /kaggle/working/PubChem10M_SMILES_BPE_450k_model_1_pampa and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
# Load your datasets
train_df = pd.read_csv('/kaggle/input/pampa-set/Train.csv')
train_df = train_df[['ID', 'SMILES', 'Permeability']]
test_df = pd.read_csv('/kaggle/input/pampa-set/Test.csv')
test_df = test_df[['ID', 'SMILES', 'Permeability']]

In [12]:
train_encodings = tokenizer(list(train_df['SMILES']), truncation=True, padding=True, max_length=325, return_tensors="pt")
test_encodings = tokenizer(list(test_df['SMILES']), truncation=True, padding=True, max_length=325, return_tensors="pt")

In [13]:
from tqdm import tqdm 
batch_size = 16 

def generate_embeddings(encodings, batch_size):
    embeddings = []
    model.eval() 
    with torch.no_grad():
        for i in tqdm(range(0, len(encodings['input_ids']), batch_size), desc="Processing batches"):
            batch = {key: val[i:i + batch_size].to(device) for key, val in encodings.items()}  
            outputs = model(**batch)
            embeddings.append(outputs.last_hidden_state)
    return torch.cat(embeddings, dim=0)


In [14]:
train_embeddings = generate_embeddings(train_encodings, batch_size)
print(train_embeddings.shape)
train_embeddings = torch.mean(train_embeddings, dim=1)
print(train_embeddings.shape)

Processing batches: 100%|██████████| 348/348 [00:42<00:00,  8.21it/s]


torch.Size([5568, 251, 768])
torch.Size([5568, 768])


In [15]:
column_names = [f'x_fine_emb_pubchem{i}' for i in range(train_embeddings.shape[1])]
embeddings_df = pd.DataFrame(data=train_embeddings.cpu().numpy(), columns=column_names)
train_data = pd.concat([train_df, embeddings_df], axis=1)

In [16]:
test_embeddings = generate_embeddings(test_encodings, batch_size)
print(test_embeddings.shape)
test_embeddings = torch.mean(test_embeddings, dim=1)
print(test_embeddings.shape)

Processing batches: 100%|██████████| 87/87 [00:10<00:00,  8.47it/s]

torch.Size([1392, 244, 768])
torch.Size([1392, 768])


In [17]:
column_names = [f'x_fine_emb_pubchem{i}' for i in range(test_embeddings.shape[1])]
embeddings_df = pd.DataFrame(data=test_embeddings.cpu().numpy(), columns=column_names)
test_data = pd.concat([test_df, embeddings_df], axis=1)

In [18]:
train_data.to_csv("/kaggle/working/Train_PubChem10M_SMILES_BPE_450k_model_1_fine_tuned_embeddings_pampa.csv",index=False)
test_data.to_csv("/kaggle/working/Test_PubChem10M_SMILES_BPE_450k_model_1_fine_tuned_embeddings_pampa.csv",index=False)

In [19]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, ExtraTreesRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression  # LogisticRegression is not used for regression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import pearsonr, spearmanr
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [20]:
train_data = pd.read_csv("/kaggle/working/Train_PubChem10M_SMILES_BPE_450k_model_1_fine_tuned_embeddings_pampa.csv")
test_data = pd.read_csv("/kaggle/working/Test_PubChem10M_SMILES_BPE_450k_model_1_fine_tuned_embeddings_pampa.csv")

In [21]:
def train_and_test_predict(models, X_train, y_train, X_test, y_test):
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    results = {}
    predictions = []  

    for model in models:
        model_name = model.__class__.__name__
        predictions_train = []
        actual_y_train = []

        test_predictions_folds = []

        

        for train_index, val_index in kf.split(X_train):
            X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
            y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

            model.fit(X_train_fold, y_train_fold)

            y_pred_fold = model.predict(X_val_fold)
            y_pred_fold = np.clip(y_pred_fold, -10, -3.9)
            predictions_train.extend(y_pred_fold)
            actual_y_train.extend(y_val_fold)

            predictions_test_fold = model.predict(X_test)
            predictions_test_fold = np.clip(predictions_test_fold, -10, -3.9)
            test_predictions_folds.append(predictions_test_fold)


        mse_train = mean_squared_error(actual_y_train, predictions_train)
        mae_train = mean_absolute_error(actual_y_train, predictions_train)
        rmse_train = np.sqrt(mse_train)
        r2_train = r2_score(actual_y_train, predictions_train)
        pearson_train, _ = pearsonr(actual_y_train, predictions_train)
        spearman_train, _ = spearmanr(actual_y_train, predictions_train)


        predictions_test_mean = np.mean(test_predictions_folds, axis=0)
        predictions_test_std = np.std(test_predictions_folds, axis=0)

        mse_test = mean_squared_error(y_test, predictions_test_mean)
        mae_test = mean_absolute_error(y_test, predictions_test_mean)
        rmse_test = np.sqrt(mse_test)
        r2_test = r2_score(y_test, predictions_test_mean)
        print(r2_test)
        pearson_test, _ = pearsonr(y_test, predictions_test_mean)
        spearman_test, _ = spearmanr(y_test, predictions_test_mean)
        
        

        predictions.append({
            'Model': model_name,
            'Y Train pred': predictions_train,
            'Y Test actual': y_test,
            'Test prediction folds': test_predictions_folds,
            'Test Predictions Mean': predictions_test_mean,
            'Test Predictions Std': predictions_test_std,

        })

        results[model_name] = {
            'Train MSE (5 fold cv)': f"{mse_train:.4f}",
            'Train MAE (5 fold cv)': f"{mae_train:.4f}",
            'Train RMSE (5 fold cv)': f"{rmse_train:.4f}",
            'Train RMSE (5 fold cv)': f"{rmse_train:.4f}",
            'Train R2 (5 fold cv)': f"{r2_train:.4f}",
            'Train PCC (5 fold cv)': f"{pearson_train:.4f}",
            'Train SCC (5 fold cv)': f"{spearman_train:.4f}",
            'Test MSE': f"{mse_test:.4f}",
            'Test MAE': f"{mae_test:.4f}",
            'Test RMSE': f"{rmse_test:.4f}",
            'Test R2': f"{r2_test:.4f}",
            'Test Pearson Correlation': f"{pearson_test:.4f}",
            'Test Spearman Correlation': f"{spearman_test:.4f}",
        }

    results_df = pd.DataFrame(results).T
    predictions_df = pd.DataFrame(predictions)

    return results_df, predictions_df



In [22]:
X_train = train_data.drop(['ID','SMILES','Permeability'],axis=1)
y_train = train_data['Permeability']
print("X_train shape: ",X_train.shape)
print("y_train shape: ",y_train.shape)
print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")

X_test = test_data.drop(['ID','SMILES','Permeability'],axis=1)
y_test = test_data['Permeability']
print("X_test shape: ",X_test.shape)
print("y_test shape: ",y_test.shape)
print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_train = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
X_test = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)
models = [
    lgb.LGBMRegressor(objective='regression',metric='rmse',boosting_type='gbdt',num_leaves=31,learning_rate=0.05,random_state=42),
    DecisionTreeRegressor(random_state=42),
    RandomForestRegressor(n_jobs=-1, random_state=42),
    GradientBoostingRegressor(random_state=42),
    AdaBoostRegressor(random_state=42),
    xgb.XGBRegressor(random_state=42),
    ExtraTreesRegressor(n_jobs=-1, n_estimators=100, random_state=42),
    LinearRegression(), 
    KNeighborsRegressor(n_neighbors=3),
    SVR(),  
    MLPRegressor(random_state=42)
]
result_df, prediction_df = train_and_test_predict(models, X_train,y_train, X_test,  y_test)
result_df

X_train shape:  (5568, 768)
y_train shape:  (5568,)
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
X_test shape:  (1392, 768)
y_test shape:  (1392,)
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.027089 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 195840
[LightGBM] [Info] Number of data points in the train set: 4454, number of used features: 768
[LightGBM] [Info] Start training from score -5.747619
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.029660 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 195840
[LightGBM] [Info] Number of data points in the train set: 4454, number of used features: 768
[LightGBM] [Info] Start training from score -5.745597
[LightGBM] [Info] Auto-choosing col-wise multi-threading

,Train MSE (5 fold cv),Train MAE (5 fold cv),Train RMSE (5 fold cv),Train R2 (5 fold cv),Train PCC (5 fold cv),Train SCC (5 fold cv),Test MSE,Test MAE,Test RMSE,Test R2,Test Pearson Correlation,Test Spearman Correlation
LGBMRegressor,0.0795,0.2072,0.2820,0.8724,0.9340,0.9181,0.2648,0.3631,0.5146,0.5816,0.7682,0.7462
DecisionTreeRegressor,0.1603,0.2965,0.4003,0.7427,0.8711,0.8429,0.2890,0.3803,0.5376,0.5433,0.7490,0.7214
RandomForestRegressor,0.0812,0.2088,0.2849,0.8697,0.9326,0.9164,0.2674,0.3644,0.5171,0.5774,0.7660,0.7443
GradientBoostingRegressor,0.0797,0.2078,0.2823,0.8721,0.9339,0.9180,0.2679,0.3662,0.5176,0.5767,0.7659,0.7433
AdaBoostRegressor,0.0927,0.2333,0.3044,0.8512,0.9237,0.9063,0.2724,0.3770,0.5219,0.5695,0.7603,0.7376
XGBRegressor,0.0937,0.2264,0.3061,0.8496,0.9220,0.9046,0.2657,0.3649,0.5154,0.5802,0.7680,0.7434
ExtraTreesRegressor,0.0825,0.2102,0.2872,0.8675,0.9314,0.9157,0.2653,0.3640,0.5151,0.5808,0.7679,0.7442
LinearRegression,0.0898,0.2224,0.2997,0.8558,0.9256,0.9130,0.2724,0.3707,0.5220,0.5695,0.7618,0.7419
KNeighborsRegressor,0.1096,0.2446,0.3311,0.8240,0.9087,0.8845,0.2774,0.3674,0.5267,0.5616,0.7590,0.7411
SVR,0.0802,0.2060,0.2833,0.8712,0.9334,0.9197,0.2645,0.3623,0.5143,0.5820,0.7690,0.7489


In [23]:
result_df

,Train MSE (5 fold cv),Train MAE (5 fold cv),Train RMSE (5 fold cv),Train R2 (5 fold cv),Train PCC (5 fold cv),Train SCC (5 fold cv),Test MSE,Test MAE,Test RMSE,Test R2,Test Pearson Correlation,Test Spearman Correlation
LGBMRegressor,0.0795,0.2072,0.2820,0.8724,0.9340,0.9181,0.2648,0.3631,0.5146,0.5816,0.7682,0.7462
DecisionTreeRegressor,0.1603,0.2965,0.4003,0.7427,0.8711,0.8429,0.2890,0.3803,0.5376,0.5433,0.7490,0.7214
RandomForestRegressor,0.0812,0.2088,0.2849,0.8697,0.9326,0.9164,0.2674,0.3644,0.5171,0.5774,0.7660,0.7443
GradientBoostingRegressor,0.0797,0.2078,0.2823,0.8721,0.9339,0.9180,0.2679,0.3662,0.5176,0.5767,0.7659,0.7433
AdaBoostRegressor,0.0927,0.2333,0.3044,0.8512,0.9237,0.9063,0.2724,0.3770,0.5219,0.5695,0.7603,0.7376
XGBRegressor,0.0937,0.2264,0.3061,0.8496,0.9220,0.9046,0.2657,0.3649,0.5154,0.5802,0.7680,0.7434
ExtraTreesRegressor,0.0825,0.2102,0.2872,0.8675,0.9314,0.9157,0.2653,0.3640,0.5151,0.5808,0.7679,0.7442
LinearRegression,0.0898,0.2224,0.2997,0.8558,0.9256,0.9130,0.2724,0.3707,0.5220,0.5695,0.7618,0.7419
KNeighborsRegressor,0.1096,0.2446,0.3311,0.8240,0.9087,0.8845,0.2774,0.3674,0.5267,0.5616,0.7590,0.7411
SVR,0.0802,0.2060,0.2833,0.8712,0.9334,0.9197,0.2645,0.3623,0.5143,0.5820,0.7690,0.7489


In [24]:
prediction_df

,Model,Y Train pred,Y Test actual,Test prediction folds,Test Predictions Mean,Test Predictions Std
0,LGBMRegressor,"[-7.049910077033714, -6.910127214879606, -6.80...",0 -7.00 1 -7.00 2 -7.00 3 ...,"[[-5.424753365201666, -6.273459929093183, -6.6...","[-5.392619380870496, -6.367866432647046, -6.53...","[0.022406440287576376, 0.07105642657620204, 0...."
1,DecisionTreeRegressor,"[-6.775620616, -7.0, -5.86, -7.0, -7.0, -6.8, ...",0 -7.00 1 -7.00 2 -7.00 3 ...,"[[-5.4, -5.86, -5.7, -6.15, -4.83, -4.44, -5.8...","[-5.314, -6.318, -6.422, -6.557569113, -4.988,...","[0.13979985693841046, 0.3411392677485252, 0.36..."
2,RandomForestRegressor,"[-6.992182820239999, -6.94636765342, -6.699097...",0 -7.00 1 -7.00 2 -7.00 3 ...,"[[-5.37645, -6.332588267519998, -6.32404446427...","[-5.370894949306001, -6.358604769251999, -6.36...","[0.027172425394116746, 0.094454450647812, 0.09..."
3,GradientBoostingRegressor,"[-6.874411109580283, -6.946725278658265, -6.83...",0 -7.00 1 -7.00 2 -7.00 3 ...,"[[-5.388275077532727, -6.413310047053667, -6.3...","[-5.37676093304538, -6.418600257134936, -6.326...","[0.04223265126634053, 0.029627695128357383, 0...."
4,AdaBoostRegressor,"[-7.212290349770136, -6.929859681286143, -6.48...",0 -7.00 1 -7.00 2 -7.00 3 ...,"[[-5.497750754252755, -6.276310286270183, -6.2...","[-5.5115760596610475, -6.414447135250141, -6.2...","[0.037523295369909476, 0.08917985482544946, 0...."
5,XGBRegressor,"[-6.9831333, -7.0333514, -6.433008, -7.004973,...",0 -7.00 1 -7.00 2 -7.00 3 ...,"[[-5.36931, -6.2559366, -6.5749793, -6.5638747...","[-5.3832016, -6.4496565, -6.515744, -6.4482, -...","[0.021501217, 0.14929686, 0.099935204, 0.11964..."
6,ExtraTreesRegressor,"[-7.018435606844999, -6.972823086060001, -6.70...",0 -7.00 1 -7.00 2 -7.00 3 ...,"[[-5.414602316169997, -6.289498690010004, -6.4...","[-5.383378059767999, -6.370238486712, -6.43966...","[0.03804167866891496, 0.07989749671403575, 0.0..."
7,LinearRegression,"[-7.127497547824292, -6.951647979332949, -6.97...",0 -7.00 1 -7.00 2 -7.00 3 ...,"[[-5.652957435067805, -6.099530049267482, -6.2...","[-5.576201168621712, -6.112872029409277, -6.30...","[0.11708583419461675, 0.037904796990071675, 0...."
8,KNeighborsRegressor,"[-6.933333333333334, -7.0, -6.746666666666667,...",0 -7.00 1 -7.00 2 -7.00 3 ...,"[[-5.45, -6.003333333333333, -6.18666666666666...","[-5.699333333333334, -5.9446666666666665, -6.1...","[0.15432145382645615, 0.07326056996168595, 0.0..."
9,SVR,"[-7.038592919453993, -6.8893048923062565, -6.8...",0 -7.00 1 -7.00 2 -7.00 3 ...,"[[-5.525350409204373, -6.1022731055871295, -6....","[-5.553101779034496, -6.131623710678831, -6.51...","[0.04185733666042138, 0.06747982411583067, 0.0..."


In [25]:
result_df.to_csv('/kaggle/working/Results_PubChem10M_SMILES_BPE_450k_model_1_fine_tuned_embeddings_pampa.csv')
prediction_df.to_csv('/kaggle/working/Prediction_data_PubChem10M_SMILES_BPE_450k_model_1_fine_tuned_embeddings_pampa.csv')

In [26]:
result_df

,Train MSE (5 fold cv),Train MAE (5 fold cv),Train RMSE (5 fold cv),Train R2 (5 fold cv),Train PCC (5 fold cv),Train SCC (5 fold cv),Test MSE,Test MAE,Test RMSE,Test R2,Test Pearson Correlation,Test Spearman Correlation
LGBMRegressor,0.0795,0.2072,0.2820,0.8724,0.9340,0.9181,0.2648,0.3631,0.5146,0.5816,0.7682,0.7462
DecisionTreeRegressor,0.1603,0.2965,0.4003,0.7427,0.8711,0.8429,0.2890,0.3803,0.5376,0.5433,0.7490,0.7214
RandomForestRegressor,0.0812,0.2088,0.2849,0.8697,0.9326,0.9164,0.2674,0.3644,0.5171,0.5774,0.7660,0.7443
GradientBoostingRegressor,0.0797,0.2078,0.2823,0.8721,0.9339,0.9180,0.2679,0.3662,0.5176,0.5767,0.7659,0.7433
AdaBoostRegressor,0.0927,0.2333,0.3044,0.8512,0.9237,0.9063,0.2724,0.3770,0.5219,0.5695,0.7603,0.7376
XGBRegressor,0.0937,0.2264,0.3061,0.8496,0.9220,0.9046,0.2657,0.3649,0.5154,0.5802,0.7680,0.7434
ExtraTreesRegressor,0.0825,0.2102,0.2872,0.8675,0.9314,0.9157,0.2653,0.3640,0.5151,0.5808,0.7679,0.7442
LinearRegression,0.0898,0.2224,0.2997,0.8558,0.9256,0.9130,0.2724,0.3707,0.5220,0.5695,0.7618,0.7419
KNeighborsRegressor,0.1096,0.2446,0.3311,0.8240,0.9087,0.8845,0.2774,0.3674,0.5267,0.5616,0.7590,0.7411
SVR,0.0802,0.2060,0.2833,0.8712,0.9334,0.9197,0.2645,0.3623,0.5143,0.5820,0.7690,0.7489


In [27]:
prediction_df

,Model,Y Train pred,Y Test actual,Test prediction folds,Test Predictions Mean,Test Predictions Std
0,LGBMRegressor,"[-7.049910077033714, -6.910127214879606, -6.80...",0 -7.00 1 -7.00 2 -7.00 3 ...,"[[-5.424753365201666, -6.273459929093183, -6.6...","[-5.392619380870496, -6.367866432647046, -6.53...","[0.022406440287576376, 0.07105642657620204, 0...."
1,DecisionTreeRegressor,"[-6.775620616, -7.0, -5.86, -7.0, -7.0, -6.8, ...",0 -7.00 1 -7.00 2 -7.00 3 ...,"[[-5.4, -5.86, -5.7, -6.15, -4.83, -4.44, -5.8...","[-5.314, -6.318, -6.422, -6.557569113, -4.988,...","[0.13979985693841046, 0.3411392677485252, 0.36..."
2,RandomForestRegressor,"[-6.992182820239999, -6.94636765342, -6.699097...",0 -7.00 1 -7.00 2 -7.00 3 ...,"[[-5.37645, -6.332588267519998, -6.32404446427...","[-5.370894949306001, -6.358604769251999, -6.36...","[0.027172425394116746, 0.094454450647812, 0.09..."
3,GradientBoostingRegressor,"[-6.874411109580283, -6.946725278658265, -6.83...",0 -7.00 1 -7.00 2 -7.00 3 ...,"[[-5.388275077532727, -6.413310047053667, -6.3...","[-5.37676093304538, -6.418600257134936, -6.326...","[0.04223265126634053, 0.029627695128357383, 0...."
4,AdaBoostRegressor,"[-7.212290349770136, -6.929859681286143, -6.48...",0 -7.00 1 -7.00 2 -7.00 3 ...,"[[-5.497750754252755, -6.276310286270183, -6.2...","[-5.5115760596610475, -6.414447135250141, -6.2...","[0.037523295369909476, 0.08917985482544946, 0...."
5,XGBRegressor,"[-6.9831333, -7.0333514, -6.433008, -7.004973,...",0 -7.00 1 -7.00 2 -7.00 3 ...,"[[-5.36931, -6.2559366, -6.5749793, -6.5638747...","[-5.3832016, -6.4496565, -6.515744, -6.4482, -...","[0.021501217, 0.14929686, 0.099935204, 0.11964..."
6,ExtraTreesRegressor,"[-7.018435606844999, -6.972823086060001, -6.70...",0 -7.00 1 -7.00 2 -7.00 3 ...,"[[-5.414602316169997, -6.289498690010004, -6.4...","[-5.383378059767999, -6.370238486712, -6.43966...","[0.03804167866891496, 0.07989749671403575, 0.0..."
7,LinearRegression,"[-7.127497547824292, -6.951647979332949, -6.97...",0 -7.00 1 -7.00 2 -7.00 3 ...,"[[-5.652957435067805, -6.099530049267482, -6.2...","[-5.576201168621712, -6.112872029409277, -6.30...","[0.11708583419461675, 0.037904796990071675, 0...."
8,KNeighborsRegressor,"[-6.933333333333334, -7.0, -6.746666666666667,...",0 -7.00 1 -7.00 2 -7.00 3 ...,"[[-5.45, -6.003333333333333, -6.18666666666666...","[-5.699333333333334, -5.9446666666666665, -6.1...","[0.15432145382645615, 0.07326056996168595, 0.0..."
9,SVR,"[-7.038592919453993, -6.8893048923062565, -6.8...",0 -7.00 1 -7.00 2 -7.00 3 ...,"[[-5.525350409204373, -6.1022731055871295, -6....","[-5.553101779034496, -6.131623710678831, -6.51...","[0.04185733666042138, 0.06747982411583067, 0.0..."
